In [1]:
from __future__ import print_function
import keras
from keras import layers
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.constraints import maxnorm
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend
from keras.models import Model
from keras.models import Sequential
from keras.utils import plot_model, np_utils
from keras.applications import mobilenet

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

import skimage
import imgaug as ia
import math
import numpy as np
import os
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import sys

Using TensorFlow backend.


In [2]:
total_sample = 0
images = []
labels = []

In [ ]:
data_path = "owned_data/"
for label in os.listdir(data_path):
    i = 0
    label_path = os.path.join(data_path, label)
    if label.startswith("."):
        continue
    for img in os.listdir(label_path):
        if img.endswith(".jpg"):
            img_filepath = os.path.join(label_path, img)
            img = cv2.imread(img_filepath)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (32, 32))
            images.append(img)
            labels.append(label)
            i += 1
    total_sample += i    
print("Total sample:", total_sample)

In [ ]:
data_path = "own_aug_data/"
for label in os.listdir(data_path):
    i = 0
    label_path = os.path.join(data_path, label)
    if label.startswith("."):
        continue
    for img in os.listdir(label_path):
        if img.endswith(".jpg"):
            img_filepath = os.path.join(label_path, img)
            img = cv2.imread(img_filepath)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (32, 32))
            images.append(img)
            labels.append(label)
            i += 1
    total_sample += i      
print("Total sample:", total_sample)

In [ ]:
data_path = "aug_data/"
for label in os.listdir(data_path):
    i = 0
    label_path = os.path.join(data_path, label)
    if label.startswith("."):
        continue
    for img in os.listdir(label_path):
        if img.endswith(".jpg"):
            img_filepath = os.path.join(label_path, img)
            img = cv2.imread(img_filepath)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (32, 32))
            images.append(img)
            labels.append(label)
            i += 1
    total_sample += i      
print("Total sample:", total_sample)

In [ ]:
images = np.array(images)
labels = np.array(labels)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(images, labels, test_size=0.2)
# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

In [ ]:
x_train.shape, y_train.shape

In [ ]:
x_val.shape, y_val.shape

In [ ]:
x_train = x_train.reshape((x_train.shape[0], 32, 32, 1)).astype('float32')
x_val = x_val.reshape((x_val.shape[0], 32, 32, 1)).astype('float32')
# x_test = x_test.reshape((x_test.shape[0], 28, 28, 1)).astype('float32')
# normalize inputs from 0-255 to 0-1
x_train = x_train / 255
x_val = x_val / 255
# x_test = x_test / 255
# one hot encode outputs
y_train = np_utils.to_categorical(y_train, num_classes=10)
y_val = np_utils.to_categorical(y_val, num_classes=10)
# y_test = np_utils.to_categorical(y_test)
input_shape = x_train.shape[1:]
num_classes = y_train.shape[1]

In [ ]:
x_train.shape

In [3]:
test_images = []
test_labels = []

In [4]:
data_path = "data/"
total_sample = 0
for label in os.listdir(data_path):
    i = 0
    label_path = os.path.join(data_path, label)
    if label.startswith("."):
        continue
    for img in os.listdir(label_path):
        if img.endswith(".jpg"):
            img_filepath = os.path.join(label_path, img)
            img = cv2.imread(img_filepath)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (32, 32))
            test_images.append(img)
            test_labels.append(label)
            i += 1
    total_sample += i    
print("Total sample:", total_sample)

FileNotFoundError: [Errno 2] No such file or directory: 'data/'

In [ ]:
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
test_images.shape, test_labels.shape

In [ ]:
test_images = test_images.reshape((test_images.shape[0], 32, 32, 1)).astype('float32')
# normalize inputs from 0-255 to 0-1
test_images = test_images / 255
# one hot encode outputs
test_labels = np_utils.to_categorical(test_labels, num_classes=10)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    vertical_flip=False)

datagen.fit(x_train)

In [ ]:
save_dir = os.path.join(os.getcwd(), 'Saved_models')
model_name = 'CNN_model_ownAugData.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

In [ ]:
batch_size = 256
epochs = 250

In [2]:
# x_train = np.subtract(x_train, 0.5)
# x_val = np.subtract(x_val, 0.5)
# test_images = np.subtract(test_images, 0.5)

# x_train = np.multiply(x_train, 2.0)
# x_val = np.multiply(x_val, 2.0)
# test_images = np.multiply(test_images, 2.0)

def lr_schedule(epoch):
    learning_rate = 1e-3
    if epoch > 180:
        learning_rate *= 0.5e-3
    elif epoch > 150:
        learning_rate *= 1e-3
    elif epoch > 120:
        learning_rate *= 1e-2
    elif epoch > 60:
        learning_rate *= 1e-1
    print('Learning rate: ', learning_rate)
    return learning_rate

In [5]:
model = keras.models.load_model("../mobilenet_model")
model.summary()

Model: "mobilenet_1.00_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 1)         0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 33, 33, 1)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 16, 16, 32)        288       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 16, 16, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 16, 16, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 16, 16, 32)  

In [6]:
model.save_weights("../mobilenet_weights.h5")

In [3]:
model = mobilenet.MobileNet(input_shape=(32,32,1), alpha=1.0, 
                            include_top=True, weights=None, 
                            input_tensor=None, pooling='max', 
                            classes=10)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()

Learning rate:  0.001
Model: "mobilenet_1.00_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 1)         0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 33, 33, 1)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 16, 16, 32)        288       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 16, 16, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 16, 16, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliz

In [ ]:
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

In [ ]:
model.load_weights("Saved_models/mobilenet_ownData_aug.h5")

In [ ]:
# Fit the model
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_val, y_val),
                        epochs=epochs, verbose=1, workers=4,
                        callbacks=callbacks)

In [ ]:
# Final evaluation of the model
scores = model.evaluate(x_test, y_test)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# Final evaluation of the model
scores = model.evaluate(test_images, test_labels)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
model.save_weights("mobilenet_ownData_ownAug_aug.h5")

In [ ]:
def calculateDistance(x1,y1,x2,y2):  
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)  
    return dist  

def image2gray(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    return img_gray

def resize_image(img):
    img_resize = cv2.resize(img, (300, 100))
    
    return img_resize

def blur_image(img, level):
    img_blur = cv2.medianBlur(img, level)
    
    return img_blur

# Fixed
def rotate_img(img):
    img_edges = cv2.Canny(img, 50, 150)
    lines = cv2.HoughLinesP(img_edges, 1, np.pi / 180, 100, minLineLength=100, maxLineGap=2500)

    angles = []
    max_dist = 0
    saved_x1, saved_y1, saved_x2, saved_y2 = 0, 0, 0, 0
    for i in range(len(lines)):
        for x1, y1, x2, y2 in lines[i]:
            if calculateDistance(x1, y1, x2, y2) > max_dist:
                max_dist = calculateDistance(x1, y1, x2, y2)
                saved_x1, saved_y1, saved_x2, saved_y2 = x1, y1, x2, y2
            
    angle = math.degrees(math.atan2(saved_y2 - saved_y1, saved_x2 - saved_x1))
    angles.append(angle)

    median_angle = np.median(angles)
    
    (h, w) = img.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, median_angle, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h))
    
    return rotated

# def rotate_img(img):
#   img_edges = cv2.Canny(img, 50, 150)
#   lines = cv2.HoughLinesP(img_edges, 1, np.pi / 180, 100, minLineLength=10, maxLineGap=250)

#   angles = []

#   for x1, y1, x2, y2 in lines[0]:
#       angle = math.degrees(math.atan2(y2 - y1, x2 - x1))
#       angles.append(angle)

#   median_angle = np.median(angles)
  
#   (h, w) = img.shape[:2]
#   center = (w // 2, h // 2)
#   M = cv2.getRotationMatrix2D(center, median_angle, 1.0)
#   rotated = cv2.warpAffine(img, M, (w, h))
  
#   return rotated

def crop_image(img):
    # convert to gray
    img_gray = image2gray(img)

    norm_img = skimage.exposure.rescale_intensity(img_gray)

    # blur image
    blurred_img = cv2.fastNlMeansDenoising(norm_img, None, 8, 7, 21)

    # rotate img
    rotated_image = rotate_img(blurred_img)

    # resize img to 300x100
    resized_img = resize_image(rotated_image)

    # crop image to 5 part
    img_height, img_width = resized_img.shape[:2]
    cropped_img = []
    ret_img = []

    for j in range(0, img_width, img_width//6):
        cropped_img.append(resized_img[0:img_height, j:j+img_width//6])

    for k, image in enumerate(cropped_img[:5]):
        # SVM
        # img = cv2.resize(image, (28, 28))
        # img = img.flatten()
        # CNN
        img = cv2.resize(image, (32, 32))
        
        img = img.reshape((1, 32, 32, 1)).astype('float32')
        img /= 255
        img = np.subtract(img, 0.5)
        img = np.multiply(img, 2.0)
        ret_img.append(img)
    
    return ret_img

In [ ]:
for f in os.listdir("image_cropped/"):
    if f.endswith(".jpg"):
        img = cv2.imread(os.path.join("image_cropped/", f))
        ia.imshow(img)
        cropped_img = crop_image(img)
        y_pred = []
        for i in range(len(cropped_img)):
            y_pred.append(model.predict(cropped_img[i]))
        pred = []
        for i in range(len(y_pred)):
            pred.append(np.argmax(y_pred[i]))
        print(pred)